# Spark, Datasets, and Structured Streaming

# Spark Overview
<!-- We'll discuss the advantages of Spark, its main features, and advantages over existing systems. -->

[Spark](http://spark.apache.org/) is a platform for distributed computation that has several great features:

1. Transparently processes data on multiple nodes in the cloud while giving the programmer an API that's little more complex than Scala's `Seq` API.
1. Resiliently handles failures by restarting processes.
1. Spills data to disk as necessary but prefers to intelligently cache in ram for faster processing.
1. Java, Scala, Python, R, and SQL APIs (although we will primarily be using the Scala one).
1. The same Spark code can run in standalone (single-node cluster for development), Hadoop, mesos, the cloud.

# Spark Wordcount Using RDD Example
<!-- We'll introuce a simple wordcount in Spark to illustrate RDDs, the basic concept of chaining operations, simple output, and lazy computation.  RDDs are still important for DStreams in Spark. -->

In order to speak about Spark Structured Streaming, we need to first introduce the basics of batch (i.e. non-streaming) jobs in Spark and highlight the main features of the batch API.  In addition, this will be useful when we look at Spark discretized streams later on.

The key feature of the batch API is an resilient distributed dataset (RDD).  These share a similar API to Scala `Seq` and transparently handle large datasets by conveniently abstracting the distributing computation load.

Below, we give a simple example of a Spark Program that counts the words in Shakespeare's Othello.  Notice how much more concise the code is compared to the typical [wordcount in MapReduce](https://hadoop.apache.org/docs/current/hadoop-mapreduce-client/hadoop-mapreduce-client-core/MapReduceTutorial.html#Example:_WordCount_v2.0) that's 133 lines of code.

While each of the commands looks like it's doing something, they're not.  It turns out that Spark has two types of chained methods: 

1. **Transformations** are methods that take an RDD and return another RDD (**chaining**) by adding a step to the query plan.  They perform no actions and are **lazy computations**, simply adding another job to the **query plan** or plan for how to execute the query.  Examples include `map`, `flatMap`, `filter`, `groupByKey`, and `reduceByKey`.
1. **Actions** are methods on RDDs that return nothing but typically trigger outputting of data.  Examples include `foreach` and `saveasTextFile`.

In [ ]:
// Load data
val lines = sc.textFile("data/othello.txt")

// Count words
val counts = (lines.flatMap(line => line.split("\\s+"))
    .map(word => (word.toLowerCase, 1))
    .reduceByKey(_ + _))
    
// Sort and print top 20
(counts.sortBy(_._2, ascending=false)
    .take(20)
    .foreach(println))

# Spark Wordcount Using Scala Example
<!-- We demonstrate how Scala Sequences have a similar API to RDDs to help illustrate Spark's simplicity. -->

Rather than using the complex MapReduce paradigm, Spark's API resembles the the Scala `Seq` API.  For example, the code below uses native Scala methods to implement a (much less powerful) word count.  The amazing thing about Spark is that it allows us distributed computation with the same level of API complexity.

In [ ]:
import scala.io.Source

// Load data
val lines = (Source.fromFile("data/othello.txt")
    .getLines
 
// Count words
val counts = (lines.flatMap(line => line.split("\\s+"))
    .toSeq
    .groupBy(_.toLowerCase)
    .mapValues(_.length))
       
// Sort and print top 20
(counts.toSeq
    .sortBy(_._2)
    .reverse
    .take(20)
    .foreach(println))

**Exercise**: Use the `filter` method on RDDs to only count the number of instances of lowercase words using Spark.  Verify your answer with the native Scala implementation using `Seq`.

# Spark and Datasets
<!-- We'll introduce Spark Datasets, which are the basis for Spark Structured Streaming.  We'll explain some of the advantages of Datasets. -->

While RDDs are great, they lack several things.
- RDDs are slow because of Java serialization (this can be sped up by using Kryo).
- Dataframes provide named columns and identical rows, like [Dataframes in R](http://www.r-tutor.com/r-introduction/data-frame) or Dataframes in [Python Pandas](http://pandas.pydata.org/).  It benefits from many advanced optimizations.  Unfortunately, we sacrifice typesafety.
- Datasets provide an API which has many of those options as before.  They have many of the same performance benefits without needing to sacrifice typesafety.

Furthermore, Datasets are the basis for Spark Structured Streaming so understanding this API is critical for understanding Spark Structured Streaming.

# Spark Wordcount Using Datasets Example
<!-- Through a wordcount example, we'll illustrate the Dataset API, its use of implicits, and its relationship to SQL. -->

The dataset API is quite close to the RDD one.  One major difference is its reliance on column names (like in SQL or Dataframes) rather than the functional access patterns.

That implies that datasets make heavy use of Scala Implicits, which we will have to import.  Implicit objects are implicitly in the context and are passed in as necessary by the compiler.  They are mostly transparent from the user and you only have to remember to import them.  To learn more, you can read the [Scala documentation](http://docs.scala-lang.org/tutorials/tour/implicit-parameters.html).  The API also uses column types which are strings prefixed by `$` or called from the method `.col`, which are imported at the same time.

In [ ]:
// Normally, run `import spark.implicits._`
// There's a namespace collision in a Spark Notebook that requires this

val sparkDummy = spark
import sparkDummy.implicits._

In [ ]:
// Load Data
val text = (spark.read
    .text("data/othello/part*")
    .as[String])

// Count words
val counts = (text.flatMap(line => line.split("\\s+"))
    .groupByKey(_.toLowerCase)
    .count)

// Display most common
counts.orderBy($"count(1)" desc).show

# Joining Data Using Spark Datasets
<!-- One of the trickiest design patterns in not just Spark but distributed computation overall is the data join.  We'll demonstrate how to do it in a simple Spark Dataset example joining user data with user transaction history. -->

So far, we have covered `map`, `flatMap`, and `filter`, representing three major operations on RDDs (which are also used in streaming).  The final operation we will cover is loading and joining structured data.

**Loading Data:** Datasets supports many nice helper functions for loading data.
1. In the code below, we infer the data schema, read from headers, and cast to a case class.
1. It also supports a SQL-like join syntax via the special `===` operator.

In [ ]:
import sys.process._

"more data/users.csv" !

In [ ]:
case class User(id: Int, name: String, email: String, country: String)
case class Transaction(userid: Int, product: String, cost: Double)

val users = (spark.read
    .option("inferSchema", "true")
    .option("header", "true")
    .csv("data/users.csv")
    .as[User])
    
val transactions = (spark.read
    .option("inferSchema", "true")
    .option("header", "true")
    .csv("data/transactions/*.csv")
    .as[Transaction])

(users.join(transactions, users.col("id") === transactions.col("userid"))
    .groupBy($"name")
    .sum("cost")
    .show)

**Exercise:** Notice that there's a user ("Edward") who did not make a purchase.  He disappeared from `transactionsByUsers` because we were doing an (inner) `join`.  Instead, do a left outer join so we have a record that he spent nothing.

# Structured Streaming Overview
<!-- Building on our foundations of Datasets, we'll dive into the conceptual model of Spark Structured Streaming and what's going on when you make a query to a Spark Structured Stream. -->

So far, we have only talked about batch operations in the form of RDDs and Datasets.  We will now introduce Spark Structured Streaming.  The easiest way to think about Spark Structured Streaming as a dataset with an infinite number of rows.  As more data is added, more rows are appended to the dataset.

![](images/structured-streaming-stream-as-a-table.png)

This interface has a number of benefits, primary among them being that the API for streaming is almost exactly the same as the API for datasets.

With Spark Structured Streaming, we define a trigger which occurs regularly (for example, every second).  When the trigger fires, more input (table rows) are appended to the "dataset".  We can then write queries (for example, wordcount) against the entire (ever growing) "dataset".  Of course, the queries are computed incrementally so that we only have to process the new data at each time point.

![](images/structured-streaming-model.png)

Finally, it's worth keeping in mind that with streaming, spark is still distributing the workload across multiple nodes.  Your data is essentailly distributed along two dimensions: time (as the data streams in) and nodes.

<!--- Images are from https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html --->

# Spark Structured Streaming Wordcount Example
<!-- We'll explore a simple Spark Structured Streaming Wordcount example to demonstrate the similarity with the Spark Dataset example. -->

Below, we have chunked Othello into multiple parts.  Structured Streaming proves a useful option for reading in each part on a separate trigger, allowing us to fake a stream from a file.

Note how similar the Structured Streaming API is to the Dataset API.

In [ ]:
// Load data
val text = (spark.readStream
    .option("maxFilesPerTrigger", 1)  // fake streaming read one file per trigger
    .text("data/othello/part*")
    .as[String])
    
// Count words
val counts = (text.flatMap(line => line.split("\\s+"))
    .groupByKey(_.toLowerCase)
    .count)
    
// Print counts
val query = (counts
    .orderBy($"count(1)" desc)
    .writeStream
    .outputMode("complete")
    .format("console")
    .start)

<img src="images/logo-text.jpg" width="20%"/>